# Dataset Introduction

This notebook is trying to replicate the [Github Rebo](https://github.com/N-Nieto/Inner_Speech_Dataset) of the `Inner Speech Classification` Dataset. Maily the tutorial notebook.

[Notebook Link](https://github.com/N-Nieto/Inner_Speech_Dataset/blob/main/Database_load_Tutorial.ipynb)

In [ ]:
#  !git clone https://github.com/N-Nieto/Inner_Speech_Dataset -quit

In [1]:
import os
import mne
import pickle
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt

from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Select_time_window, Transform_for_classificator, Split_trial_in_time

np.random.seed(23)

mne.set_log_level(verbose='warning') #to avoid info at terminal
warnings.filterwarnings(action = "ignore", category = DeprecationWarning )
warnings.filterwarnings(action = "ignore", category = FutureWarning )

## Load Data

In [7]:
### Hyperparameters

datatype = "EEG"     # Data Type
fs = 256             # Sampling rate
t_start = 1.5        # Select the useful par of each trial. Time in seconds
t_end = 3.5
N_S = 1              # Subject number [1 to 10]

In [8]:
#@title Data extraction and processing
# The root dir have to point to the folder that cointains the database
root_dir = "dataset/inner-speech-recognition"

# Load all trials for a sigle subject
X, Y = Extract_data_from_subject(root_dir, N_S, datatype)

# Cut usefull time. i.e action interval
X = Select_time_window(X = X, t_start = t_start, t_end = t_end, fs = fs)


print("Data shape: [trials x channels x samples]")
print(X.shape) # Trials, channels, samples

print("Labels shape")
print(Y.shape) # Time stamp, class , condition, session

Data shape: [trials x channels x samples]
(500, 128, 512)
Labels shape
(500, 4)


## Create the different groups for a classifier. A group is created with one condition and one class.

In [10]:
Conditions = [["Inner"],["Inner"]]    # Conditions to compared
Classes    = [  ["Up"] ,["Down"] ]    # The class for the above condition

# Transform data and keep only the trials of interes
X , Y =  Transform_for_classificator(X, Y, Classes, Conditions)

print("Final data shape: ",X.shape)
print("Final labels shape: ",Y.shape)

Final data shape:  (100, 128, 512)
Final labels shape:  (100,)
